In [6]:
from queries import googleQuery, getDataFromGoogleQuery
from pymongo import MongoClient
client = MongoClient()
db = client['companies']

In [6]:
#Obtain starbucks in San Franciso
city = 'San Francisco'
res = googleQuery(f'Starbucks in {city}')
db['starbucks'].insert_many(getDataFromGoogleQuery(res.json(),city))
pagetoken = res.json()['next_page_token']
res = googleQuery(f'Starbucks in {city}',pagetoken)
db['starbucks'].insert_many(getDataFromGoogleQuery(res.json(),city))
pagetoken = res.json()['next_page_token']
res = googleQuery(f'Starbucks in {city}',pagetoken)
db['starbucks'].insert_many(getDataFromGoogleQuery(res.json(),city))

We have a github token:  AIza


In [2]:
#Obtain startups in San Francisco
import pandas as pd
df_startups = pd.read_csv('sanFrancisco.csv')
df_startups = df_startups[df_startups['Location']=='San Francisco']

In [7]:
from queries import getDocFromGoogleQuery

def insertStartUpToMongo(serie):
    db['startups'].insert_one({'name':serie.Name, 
                               'Number of employees':serie.Employees, 
                               'Total raised':s['Total Raised']})
    print(serie.Name + ' San Francisco')
    res = googleQuery(serie.Name + ' San Francisco').json()
    if not(res['status'] == 'ZERO_RESULTS'):
        doc = getDocFromGoogleQuery(res['results'][0],'San Francisco')
        db['startups'].update_one({'name':serie.Name},{'$set':doc})

df_startups.apply(insertToMongo, axis = 1)

In [1]:
#Create schools collection with a database containing schools in San Francisco
import json
with open('Schools.geojson') as f:
    data = json.load(f)
    
print(len(data['features']), 'Schools in San Fransisco')
from database import insertSchoolToMongo

for e in data['features']:
    insertSchoolToMongo(db,e)

In [12]:
import pymongo

for col in ['schools','starbucks','rentings','startups']:
    db[col].create_index([("location", pymongo.GEOSPHERE)])


In [48]:
sphericalQuery = lambda long,lat,radius:{'location': {'$geoWithin': {'$centerSphere':[[long,lat], radius/6371]}}}
list(db['schools'].find(sphericalQuery(o['location']['coordinates'][0],o['location']['coordinates'][1],0.225)))

#From 0 to 500 meters punctuation

[{'_id': ObjectId('5ddcb25e7f05b4bc17a61dd1'),
  'type': 'Feature',
  'campus_address': '657 MERCHANT ST, SAN FRANCISCO, CA 94111',
  'campus_name': 'Chinese Education Center Elementary School',
  'grade_range': 'K-5',
  'category': 'USD Grades K-5',
  'location': {'type': 'Point', 'coordinates': [-122.40406, 37.794769]}}]

In [136]:
available_offices = list(db['rentings'].find({'city':'San Francisco','address1':{'$ne':''}}))

def findMinDistance(place,office):
    #Find closest school and starbucks
    c = office['location']['coordinates']
    meters = 0
    found = 0
    while meters < 1000 and not found:
        meters += 5
        found = len(list(db[place].find(sphericalQuery(c[0],c[1],meters/1000))))
    return meters

def findTenStartups(office):
    #Find 10 surrounding startups
    c = office['location']['coordinates']
    meters = 0
    found = False
    while meters < 1000 and not found:
        meters += 5
        found = len(list(db['startups'].find(sphericalQuery(c[0],c[1],meters/1000)))) >= 10
    return meters
    
findTenStartups(o)

620

In [137]:
for office in available_offices:
    db['rentings'].update_one(office,{'$set':
                                     {'Closest 10 startups':findTenStartups(office),
                                     'Closest Starbucks': findMinDistance('starbucks',office),
                                     'Closest school': findMinDistance('schools',office)}})

In [82]:
len(list(db['rentings'].find()))

53

In [142]:
def getRating(*distances, ponderations=[50,25,25]):
    scores = []
    for d in distances:
        scores.append((1000-d)/100)
    return sum([s*p/100 for s,p in zip(scores,ponderations)])
        
for office in available_offices:
    db['rentings'].update_one(office,{'$set':
                                     {'Rating': getRating(office['Closest 10 startups'],
                                                          office['Closest Starbucks'],
                                                          office['Closest school'])}})

In [8]:
final_renting = db['rentings'].find().sort('Rating', -1)[0]
print('Best renting office is found in ',final_renting['address1'])
final_renting 


Best renting office is found in  425 Market Street , Suite 2200


{'_id': ObjectId('5dd71e6928d15205f49de2aa'),
 'description': None,
 'address1': '425 Market Street , Suite 2200',
 'address2': '',
 'zip_code': '94105',
 'city': 'San Francisco',
 'state_code': 'CA',
 'country_code': 'USA',
 'company': ObjectId('52cdef7d4bab8bd675298a8a'),
 'location': {'type': 'Point', 'coordinates': [-122.398126, 37.791128]},
 'lat&long': [37.791128, -122.398126],
 'Closest 10 startups': 460,
 'Closest Starbucks': 55,
 'Closest school': 220,
 'Rating': 7.0125}

In [21]:
#Map with tech startups and available offices in San Francisco
import folium

#latitude, longitude
m =folium.Map(location=[37.773972, -122.431297],zoom_start=13)
for place in ['schools','starbucks','startups']:
    if place == 'schools':
        for p in db['schools'].find({'location':{'$exists':True}}):
            folium.Marker(p['location']['coordinates'][::-1],radius=2,
                          icon=folium.Icon(icon='android',prefix='fa',color='darkblue')).add_to(m)

    elif place == 'starbucks':
        for p in db['starbucks'].find({'location':{'$exists':True}}):
            folium.Marker(p['location']['coordinates'][::-1],radius=2,
                        icon=folium.Icon(icon='home',prefix='fa',color='red')).add_to(m)
    elif place == 'startups':
        for p in db['startups'].find({'location':{'$exists':True}}):
            folium.Marker(p['location']['coordinates'][::-1],radius=2,
                        icon=folium.Icon(icon='coffee',prefix='fa',color='darkgreen')).add_to(m)

folium.Marker(final_renting['location']['coordinates'][::-1], radius=2,color = 'red').add_to(m)

m.save('startups_map')
m

{'type': 'Point', 'coordinates': [-122.439858, 37.785936]}
{'type': 'Point', 'coordinates': [-122.417347, 37.7770595]}
{'type': 'Point', 'coordinates': [-122.453194, 37.786261]}
{'type': 'Point', 'coordinates': [-122.420425, 37.806565]}
{'type': 'Point', 'coordinates': [-122.446293, 37.775737]}
{'type': 'Point', 'coordinates': [-122.393893, 37.767102]}
{'type': 'Point', 'coordinates': [-122.40322, 37.787404]}
{'type': 'Point', 'coordinates': [-122.39485, 37.796647]}
{'type': 'Point', 'coordinates': [-122.430395, 37.797644]}
{'type': 'Point', 'coordinates': [-122.4106957, 37.7871735]}
{'type': 'Point', 'coordinates': [-122.40576, 37.790431]}
{'type': 'Point', 'coordinates': [-122.434666, 37.761019]}
{'type': 'Point', 'coordinates': [-122.40377, 37.792183]}
{'type': 'Point', 'coordinates': [-122.4076337, 37.7841086]}
{'type': 'Point', 'coordinates': [-122.395103, 37.791511]}
{'type': 'Point', 'coordinates': [-122.408456, 37.789146]}
{'type': 'Point', 'coordinates': [-122.467327, 37.73973